# Regression Model A: Field 3

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 3 (F3)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E3" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E3" ),]
dE4=d4[which(d4$Estate=="E3" ),]
dE5=d5[which(d5$Estate=="E3" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Regression

Fit the data using multiple regression for all variables. 

In [7]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=dg1)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-44.635 -19.945  -4.094  18.330  73.546 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.302e+02  1.887e+01   6.898 5.17e-08 ***
rainfall      -2.774e-03  7.419e-03  -0.374  0.71075    
foliar        -4.378e+00  1.430e+00  -3.061  0.00422 ** 
prun.time     -2.600e+00  3.885e+00  -0.669  0.50763    
prun.cost      3.953e-03  6.005e-03   0.658  0.51470    
manuring.time -1.541e+01  6.507e+00  -2.368  0.02356 *  
manuring.cost  1.594e-03  9.525e-04   1.674  0.10312    
pd.time       -1.538e+00  6.243e+00  -0.246  0.80683    
pd.cost        3.376e-03  6.302e-03   0.536  0.59555    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 30.4 on 35 degrees of freedom
Multiple R-squared:  0.294,	Adjusted R-squared:  0.1326 
F

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [8]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,341.2067,341.2067,0.36930212,0.54730803
foliar,1,6795.9832,6795.9832,7.35557446,0.01029486
prun.time,1,659.6122,659.6122,0.71392565,0.40388273
prun.cost,1,145.6006,145.6006,0.15758961,0.69379589
manuring.time,1,2719.8603,2719.8603,2.94381762,0.09504782
manuring.cost,1,2525.7728,2525.7728,2.73374869,0.10719174
pd.time,1,11.3419,11.3419,0.01227581,0.91241084
pd.cost,1,265.1504,265.1504,0.28698330,0.59554837
Residuals,35,32337.2993,923.9228,NA,NA


### Using stepwise

In [9]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=308.39
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- pd.time        1      56.1 32393 306.47
- rainfall       1     129.1 32466 306.57
- pd.cost        1     265.2 32602 306.75
- prun.cost      1     400.3 32738 306.93
- prun.time      1     414.0 32751 306.95
<none>                       32337 308.39
- manuring.cost  1    2588.0 34925 309.78
- manuring.time  1    5178.8 37516 312.93
- foliar         1    8656.7 40994 316.83

Step:  AIC=306.47
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.cost

                Df Sum of Sq   RSS    AIC
- rainfall       1     121.2 32515 304.63
- pd.cost        1     220.4 32614 304.76
- prun.cost      1     351.3 32745 304.94
- prun.time      1     429.1 32823 305.05
<none>                       32393 306.47
- manuring.cost  1    2533.8 34927 307.78
- manuring.time  1    5133.1 37526 310.94



Call:
lm(formula = yield ~ foliar + manuring.time + manuring.cost, 
    data = dg1)

Residuals:
   Min     1Q Median     3Q    Max 
-45.12 -24.35  -4.14  19.24  67.47 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.232e+02  9.827e+00  12.541 1.93e-15 ***
foliar        -4.102e+00  1.315e+00  -3.118  0.00336 ** 
manuring.time -1.581e+01  5.930e+00  -2.666  0.01102 *  
manuring.cost  1.702e-03  8.399e-04   2.026  0.04949 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.85 on 40 degrees of freedom
Multiple R-squared:  0.2731,	Adjusted R-squared:  0.2186 
F-statistic: 5.009 on 3 and 40 DF,  p-value: 0.004833


       foliar manuring.time manuring.cost 
     1.033605      1.470073      1.429983 



Call:
lm(formula = yield ~ foliar + manuring.time + manuring.cost, 
    data = dg1)

Residuals:
   Min     1Q Median     3Q    Max 
-45.12 -24.35  -4.14  19.24  67.47 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.232e+02  9.827e+00  12.541 1.93e-15 ***
foliar        -4.102e+00  1.315e+00  -3.118  0.00336 ** 
manuring.time -1.581e+01  5.930e+00  -2.666  0.01102 *  
manuring.cost  1.702e-03  8.399e-04   2.026  0.04949 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.85 on 40 degrees of freedom
Multiple R-squared:  0.2731,	Adjusted R-squared:  0.2186 
F-statistic: 5.009 on 3 and 40 DF,  p-value: 0.004833


## Prediction

Predict with the resample values.

In [10]:
set.seed(123)
dg3=sample(1:nrow(dg1),12)
dg4=dg1[dg3,]
estimate=predict(selectedMod,newdata=dg4)
estimate
sum.estimate=sum(estimate)
dg5=data.frame(estimate,sum.estimate)

13        34        18        37        38         2        21        43 
123.23125  86.31150 112.69724 122.52101 105.02783  99.64594  90.41370  98.61809 
       20        16        33        15 
 80.04435 110.92467  83.91725 115.02686